In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binary-classified-ciciot-2023-dataset/EqualBinaryData_CICIoT_2023.csv
/kaggle/input/binary-classified-ciciot-2023-dataset/Attack_baised_data.csv
/kaggle/input/binary-classified-ciciot-2023-dataset/Normal_baised_data.csv


In [3]:
import pandas as pd 
import numpy as np 

df = pd.read_csv("/kaggle/input/binary-classified-ciciot-2023-dataset/EqualBinaryData_CICIoT_2023.csv")
print(df.head())

   Unnamed: 0  Header_Length  Protocol Type  Time_To_Live        Rate  \
0        2996           26.4              6          63.6  504.310981   
1        9299           27.2              6         101.0   48.393117   
2       13869           29.6              6         153.1  135.512578   
3        1084           32.0              6          80.2  182.292090   
4        8499           28.0              6          75.9   46.047128   

   fin_flag_number  syn_flag_number  rst_flag_number  psh_flag_number  \
0              0.0              0.0              0.0              0.0   
1              0.0              0.0              0.0              0.5   
2              0.0              0.0              0.0              0.2   
3              0.0              0.0              0.0              0.1   
4              0.2              0.1              0.0              0.2   

   ack_flag_number  ...  Tot sum  Min   Max     AVG          Std  Tot size  \
0              0.8  ...    11373   66  1514 

In [4]:
from sklearn.model_selection import  train_test_split

X = df.drop('Label', axis = 1)
# print(X)
y = df['Label']
# print(y)
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [5]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# ----- STEP 1: COPY X_train, X_test -----
X_train_clean = X_train.copy()
X_test_clean  = X_test.copy()

# ----- STEP 2: Replace ±inf → NaN -----
X_train_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test_clean.replace([np.inf, -np.inf], np.nan, inplace=True)

# ----- STEP 3: SimpleImputer (median) -----
imputer = SimpleImputer(strategy="median")

# Fit only on training data (important!)
X_train_imputed = imputer.fit_transform(X_train_clean)
X_test_imputed  = imputer.transform(X_test_clean)

# ----- STEP 4: StandardScaler -----
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled  = scaler.transform(X_test_imputed)

print("All preprocessing DONE with SimpleImputer + StandardScaler")


All preprocessing DONE with SimpleImputer + StandardScaler


LOGISTIC REGRESSION 

In [6]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=2000, class_weight='balanced', random_state=0)
lr.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=2000, random_state=0)

In [7]:
from sklearn.metrics import accuracy_score
y_pred = lr.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 99.49%


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

scores = cross_validate(estimator=lr, X=X_train_scaled, y=y_train,
                        cv=5, scoring='roc_auc', 
                        return_train_score=True, return_estimator=True)




In [10]:
print(scores['train_score'])
print(scores['test_score'])

[0.99807323 0.99805845 0.99808353 0.99808481 0.99809217]
[0.99810476 0.99812916 0.99804741 0.9980498  0.9980307 ]


In [13]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

print("\nTraining Linear SVM...")
svm_lin = LinearSVC(class_weight="balanced")
svm_lin.fit(X_train_scaled, y_train)
y_pred_svm_lin = svm_lin.predict(X_test_scaled)

print(classification_report(y_test, y_pred_svm_lin))



Training Linear SVM...


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      0.99      0.99    119385
           1       0.99      1.00      0.99    119780

    accuracy                           0.99    239165
   macro avg       0.99      0.99      0.99    239165
weighted avg       0.99      0.99      0.99    239165



In [ ]:
from sklearn.svm import SVC

print("\nTraining RBF SVM...")
svm_rbf = SVC(kernel="rbf", class_weight="balanced", probability=True)
svm_rbf.fit(X_train_scaled, y_train)
y_pred_svm_rbf = svm_rbf.predict(X_test_scaled)

print(classification_report(y_test, y_pred_svm_rbf))
all_models_results["SVM RBF"] = y


Training RBF SVM...


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

print("\nTraining KNN (k=5)...")
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

print(classification_report(y_test, y_pred_knn))
all_models_results["KNN"] = y_pred_knn

In [ ]:
from sklearn.naive_bayes import GaussianNB

print("\nTraining Gaussian Naive Bayes...")
nb = GaussianNB()
nb.fit(X_train_scaled, y_train)
y_pred_nb = nb.predict(X_test_scaled)

print(classification_report(y_test, y_pred_nb))
all_models_results["GaussianNB"] = y_pred_nb


In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("\nTraining RandomForest...")
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=25,
    n_jobs=-1,
    class_weight="balanced_subsample",
    random_state=42
)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

print(classification_report(y_test, y_pred_rf))
all_models_results["RandomForest"] = y_pred_rf


In [ ]:
from xgboost import XGBClassifier

print("\nTraining XGBoost...")
xgb = XGBClassifier(
    n_estimators=250,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss"
)
xgb.fit(X_train_scaled, y_train)
y_pred_xgb = xgb.predict(X_test_scaled)

print(classification_report(y_test, y_pred_xgb))
all_models_results["XGBoost"] = y_pred_xgb


In [ ]:
try:
    import lightgbm as lgb

    print("\nTraining LightGBM...")
    lgbm = lgb.LGBMClassifier(
        n_estimators=400,
        boosting_type='gbdt',
        random_state=42
    )
    lgbm.fit(X_train_scaled, y_train)
    y_pred_lgbm = lgbm.predict(X_test_scaled)

    print(classification_report(y_test, y_pred_lgbm))
    all_models_results["LightGBM"] = y_pred_lgbm

except Exception as e:
    print("LightGBM not installed:", e)


In [ ]:
try:
    from catboost import CatBoostClassifier

    print("\nTraining CatBoost...")
    cat = CatBoostClassifier(
        iterations=300,
        depth=6,
        learning_rate=0.1,
        loss_function="Logloss",
        verbose=0
    )
    cat.fit(X_train_scaled, y_train)
    y_pred_cat = cat.predict(X_test_scaled)

    print(classification_report(y_test, y_pred_cat))
    all_models_results["CatBoost"] = y_pred_cat

except Exception as e:
    print("CatBoost not installed:", e)


In [ ]:
try:
    from catboost import CatBoostClassifier

    print("\nTraining CatBoost...")
    cat = CatBoostClassifier(
        iterations=300,
        depth=6,
        learning_rate=0.1,
        loss_function="Logloss",
        verbose=0
    )
    cat.fit(X_train_scaled, y_train)
    y_pred_cat = cat.predict(X_test_scaled)

    print(classification_report(y_test, y_pred_cat))
    all_models_results["CatBoost"] = y_pred_cat

except Exception as e:
    print("CatBoost not installed:", e)
